In [1]:
from collections import deque
from itertools import islice, tee

# current implementation
def minibelt(it, size=2):
    it = iter(it)
    w = deque(islice(it, size), size)
    yield w
    for i in it:
        w.append(i)
        yield w

        
# proposed change
def pushed(it, size=2):
    it = iter(it)
    w = deque(islice(it, size), size)
    yield tuple(w)
    for i in it:
        w.append(i)
        yield tuple(w)

        
#itertools 2.3 docs
def docs(it, size=2):
    it = iter(it)
    w = tuple(islice(it, size))
    if len(w) == size:
        yield w   
    for i in it:
        w = w[1:] + (i,)
        yield w
        

# whatever this is
def hotmess(it, size=2):
    # alias zip to izip in py2
    # py3.3 + can yield from for extra wtf
    return zip(*(islice(it, s, None) for s, it in enumerate(tee(it, size))))
        
# consume iterator at C-speed
def consume(it):
    deque(it, maxlen=0)

In [2]:
test_range = range(100000)

In [3]:
%timeit consume(minibelt(test_range, size=5))

100 loops, best of 3: 13 ms per loop


In [4]:
%timeit consume(pushed(test_range, size=5))

10 loops, best of 3: 35.1 ms per loop


In [5]:
%timeit consume(docs(test_range, size=5))

10 loops, best of 3: 21 ms per loop


In [6]:
%timeit consume(hotmess(test_range, size=5))

100 loops, best of 3: 8.49 ms per loop
